# CS 1656 – Introduction to Data Science

## Instructor: Alexandros Labrinidis
## Teaching Assistants: Evangelos Karageorgos, Modhumonty Das

### Additional credits: Tahereh Arabghalizi, Zuha Agha, Anatoli Shein, Phuong Pham

## Recitation 3: Data Analysis with Pandas
---
So far we have encountered basic data manipulation with pandas Dataframes including row and column selection, boolean indexing, working with missing values, groupby and aggregate functions such as mean(). But there are many other powerful data manipulation and analysis techniques available in pandas. In this recitation, we will learn some more advanced ways for data anylsis in Python using Dataframes.

Begin by importing pandas package.

In [ ]:
import pandas as pd

Next load the dataset that we will be playing arround with.

In [9]:
df = pd.read_csv('coffee-chain.csv')
df.head()

,Area Code,Market,Market Size,Product,Product Line,Product Type,State,Type,Inventory,Budget COGS,Budget Margin,Budget Profit,Budget Sales,COGS,Margin,Marketing,Profit,Sales,Total Expenses
0,985,South,Small Market,Colombian,Beans,Coffee,Louisiana,Regular,845,50,90,70,140,49,71,13,68,128,25
1,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,540,80,110,70,190,94,120,31,114,228,43
2,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,552,90,120,80,210,101,130,33,126,246,45
3,985,South,Small Market,Decaf Irish Cream,Beans,Coffee,Louisiana,Decaf,851,70,90,80,160,48,70,13,67,126,25
4,985,South,Small Market,Lemon,Leaves,Herbal Tea,Louisiana,Decaf,599,60,80,30,140,67,83,25,37,160,58


Let's get the subset of the dataframe we need.

In [10]:
df_small = df[['Area Code','Market', 'Market Size', 'Product', 'Product Line', 'Product Type', 'State', 'Type', 'Profit','Total Expenses']].copy()
df_small.head()

,Area Code,Market,Market Size,Product,Product Line,Product Type,State,Type,Profit,Total Expenses
0,985,South,Small Market,Colombian,Beans,Coffee,Louisiana,Regular,68,25
1,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,114,43
2,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,126,45
3,985,South,Small Market,Decaf Irish Cream,Beans,Coffee,Louisiana,Decaf,67,25
4,985,South,Small Market,Lemon,Leaves,Herbal Tea,Louisiana,Decaf,37,58


## Slicing & Indexing
What we saw above was slicing. Slicing uses the [] operator selects a set of rows and/or columns from a DataFrame.

**Slicing rows**

To slice out a set of rows, you use the following syntax: data[start:stop]. When slicing in pandas the start bound is included in the output.

In [11]:
df_small[0:3]

,Area Code,Market,Market Size,Product,Product Line,Product Type,State,Type,Profit,Total Expenses
0,985,South,Small Market,Colombian,Beans,Coffee,Louisiana,Regular,68,25
1,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,114,43
2,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,126,45


**Slicing vs Copying**

We might have thought that we were creating a fresh copy of df_small when we did slicing. However the statement y = x doesn’t create a copy of our DataFrame. It creates a new variable y that refers to the same object x refers to. This means that there is only one object (the DataFrame), and both x and y refer to it. To create a fresh copy of the DataFrame you can use the syntax y=x.copy(). We will see the effect of slicing but not copying in later steps.

** Indexing **

We can select specific ranges of our data in both the row and column directions using either label or integer-based indexing.

* loc: indexing via labels or integers or mixed. Referemces rows uning the index.
* iloc: indexing via integers only. References rows using 0-based numbering.

To select a subset of rows AND columns from our DataFrame, we can use the iloc method. For example,

In [12]:
df_small.loc[0:3, 'Market': 'Product']

,Market,Market Size,Product
0,South,Small Market,Colombian
1,South,Small Market,Chamomile
2,South,Small Market,Chamomile
3,South,Small Market,Decaf Irish Cream


In [13]:
df_small.iloc[0:4, 1:4]

,Market,Market Size,Product
0,South,Small Market,Colombian
1,South,Small Market,Chamomile
2,South,Small Market,Chamomile
3,South,Small Market,Decaf Irish Cream


Notice that indexing in loc is inclusive whereas indexing in iloc is exlusive of the end index

## Statistical Techniques
### Cross-tabulation
Cross tabultaion computes a frequency table of two or more factors. Let's start by making a cross-tab with two variables first.

In [14]:
df_crosstab = pd.crosstab(df_small["Market"],df_small["Market Size"],margins=False)
df_crosstab

Market Size,Major Market,Small Market
Market,,
Central,696,648
East,552,336
South,168,504
West,288,1056


Let'c check the type of the cross-tab

In [15]:
type(df_crosstab)

pandas.core.frame.DataFrame

Now let's check the value counts of one of our cross-tab's dimensions and see if the totals match?

In [16]:
pd.value_counts(df_small['Market Size'])

/tmp/ipython-input-2391164392.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(df_small['Market Size'])


,count
Market Size,
Small Market,2544
Major Market,1704


Now let's make a cross-tab with three variables.

In [17]:
pd.crosstab(df["Product Type"], [df["Market"],df["Market Size"]],margins=True)

Market            Central                      East                     South  \
Market Size  Major Market Small Market Major Market Small Market Major Market   
Product Type                                                                    
Coffee                192          192           96           72           48   
Espresso              144          144          144           96           72   
Herbal Tea            192          144          144           72           48   
Tea                   168          168          168           96            0   
All                   696          648          552          336          168   

Market                            West                All  
Market Size  Small Market Major Market Small Market        
Product Type                                               
Coffee                144           72          240  1056  
Espresso              216           72          288  1176  
Herbal Tea            144           72          240  1056  
Tea                     0           72          288   960  
All                   504          288         1056  4248

### Binning Data

We can bin our data into categories by specifying bin widths. Let's define equal width bins as shown below. The bins array specifies 4 bins from -800 to -400, -400 to 0, 0 to 400, 400 to 800. We will also specify a group names to assign as labels to each of our bins later.

In [18]:
bins = [-800,-400, 0, 400, 800]
group_names = ['Low', 'Okay', 'Good', 'Great']

Now lets bin the data into the categories and add it as a column to the dataframe

In [19]:
df_small['Categories'] = pd.cut(df_small['Profit'], bins=bins, labels=group_names)
df_small.head(20)

,Area Code,Market,Market Size,Product,Product Line,Product Type,State,Type,Profit,Total Expenses,Categories
0,985,South,Small Market,Colombian,Beans,Coffee,Louisiana,Regular,68,25,Good
1,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,114,43,Good
2,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,126,45,Good
3,985,South,Small Market,Decaf Irish Cream,Beans,Coffee,Louisiana,Decaf,67,25,Good
4,985,South,Small Market,Lemon,Leaves,Herbal Tea,Louisiana,Decaf,37,58,Good
5,985,South,Small Market,Decaf Irish Cream,Beans,Coffee,Louisiana,Decaf,87,26,Good
6,985,South,Small Market,Lemon,Leaves,Herbal Tea,Louisiana,Decaf,43,58,Good
7,985,South,Small Market,Chamomile,Leaves,Herbal Tea,Louisiana,Decaf,48,26,Good
8,985,South,Small Market,Caffe Mocha,Beans,Espresso,Louisiana,Regular,61,35,Good
9,985,South,Small Market,Caffe Latte,Beans,Espresso,Louisiana,Regular,4,81,Good


To find out the value counts for each bin of category, we can use value_counts like we did earlier.

In [20]:
pd.value_counts(df_small['Categories'])

/tmp/ipython-input-2312271721.py:1: FutureWarning: pandas.value_counts is deprecated and will be removed in a future version. Use pd.Series(obj).value_counts() instead.
  pd.value_counts(df_small['Categories'])


,count
Categories,
Good,3648
Okay,544
Great,40
Low,16


The result is a Series with 4 values, one for every category. Notice that when printing it, we get both the category name and the value for each category. This is because the series isn't just a list of values. Every value is matched to a category, so its more like an ordered dictionary than a list. The category is the index to the values. If you just selected a column from the original dataframe you would get a Series with the row id as the index, so you would get value 0, value 1 and so on. Using operations like value_counts or group_by, you get series and dataframes that use the groups as the index.

### Quantiles

Pandas allows an easy way of computing percentiles or quartiles. Let's first specify the quantiles we want to calculate,

In [21]:
quants = [0.0, 0.05, 0.25, 0.5, 0.75, 0.95, 1.0]

To compute the quantiles of Profit and Total Expenses,

In [22]:
q = df_small[['Profit','Total Expenses']].quantile(quants)
q

,Profit,Total Expenses
0.00,-638.0,10.0
0.05,-13.0,17.0
0.25,17.0,33.0
0.50,40.0,46.0
0.75,92.0,65.0
0.95,232.0,125.0
1.00,778.0,190.0


The result here is a dataframe that uses the quantiles as the index of the rows.

### Groupby & Apply

Groupby allows grouping or clustering the dataframe by a particular categorical attribute.
Apply can be used to apply a function to a group or the entire dataframe. Let's first define the function that we want to apply,

In [23]:
def get_stats(group):
    print(type(group))
    x= {'min': group.min(), 'max': group.max(), 'count': group.count(), 'mean': group.mean(), 'sum': group.sum()}
    print(x)
    return x

This can be applied to a Dataframe or a grouping of the dataframe as shown below

In [24]:
df_group = df_small['Profit'].groupby(df_small['Categories']).apply(get_stats)
df_group

<class 'pandas.core.series.Series'>
{'min': -638, 'max': -404, 'count': np.int64(16), 'mean': np.float64(-510.5625), 'sum': np.int64(-8169)}
<class 'pandas.core.series.Series'>
{'min': -392, 'max': 0, 'count': np.int64(544), 'mean': np.float64(-45.630514705882355), 'sum': np.int64(-24823)}
<class 'pandas.core.series.Series'>
{'min': 1, 'max': 397, 'count': np.int64(3648), 'mean': np.float64(74.51452850877193), 'sum': np.int64(271829)}
<class 'pandas.core.series.Series'>
{'min': 402, 'max': 778, 'count': np.int64(40), 'mean': np.float64(517.65), 'sum': np.int64(20706)}


/tmp/ipython-input-1155890000.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df_group = df_small['Profit'].groupby(df_small['Categories']).apply(get_stats)


Categories       
Low         min        -638.000000
            max        -404.000000
            count        16.000000
            mean       -510.562500
            sum       -8169.000000
Okay        min        -392.000000
            max           0.000000
            count       544.000000
            mean        -45.630515
            sum      -24823.000000
Good        min           1.000000
            max         397.000000
            count      3648.000000
            mean         74.514529
            sum      271829.000000
Great       min         402.000000
            max         778.000000
            count        40.000000
            mean        517.650000
            sum       20706.000000
Name: Profit, dtype: float64


The result is a compound Series that has the category as the index, and each value is a dictionary. We can unwind those dictionaries and create a dataframe by using the unstack() function as shown below.

In [25]:
df_group.unstack()

,min,max,count,mean,sum
Categories,,,,,
Low,-638.0,-404.0,16.0,-510.562500,-8169.0
Okay,-392.0,0.0,544.0,-45.630515,-24823.0
Good,1.0,397.0,3648.0,74.514529,271829.0
Great,402.0,778.0,40.0,517.650000,20706.0


### Sorting

Pandas allows nested sorting over mutliple columns of the Dataframe easily as shown below.

In [26]:
data_sorted = df_small.sort_values(['Total Expenses', 'Profit'], ascending=False)
data_sorted[['Total Expenses','Profit']].head(20)

,Total Expenses,Profit
959,190,49
2334,189,50
2352,189,-284
3432,181,-266
966,180,45
2224,180,45
632,178,370
1429,178,370
631,178,368
1605,178,368


## Tasks
For your tasks, use the data file bank-data.csv.

In [27]:
df = pd.read_csv('bank-data.csv')
df.head()

,id,age,sex,region,income,married,children,car,save_act,current_act,mortgage,pep
0,ID12101,48,FEMALE,INNER_CITY,17546.0,NO,1,NO,NO,NO,NO,YES
1,ID12102,40,MALE,TOWN,30085.1,YES,3,YES,NO,YES,YES,NO
2,ID12103,51,FEMALE,INNER_CITY,16575.4,YES,0,YES,YES,YES,NO,NO
3,ID12104,23,FEMALE,TOWN,20375.4,YES,3,NO,NO,YES,NO,NO
4,ID12105,57,FEMALE,RURAL,50576.3,YES,0,NO,YES,NO,NO,NO


**Task 1**
Compute the mean income of males versus females. Return a Series of mean income as the values for all sexes.

In [33]:
df = pd.read_csv("bank-data.csv")

mean_income_by_sex = df.groupby("sex")["income"].mean()
mean_income_by_sex


,income
sex,
FEMALE,27831.368233
MALE,27216.694200


**Task 2**
Create a cross-tab of save_acct and mortgage. Include the margins.

In [32]:
df = pd.read_csv("bank-data.csv")

ct = pd.crosstab(df["save_act"], df["mortgage"], margins=True)
ct


mortgage,NO,YES,All
save_act,,,
NO,121,65,186
YES,270,144,414
All,391,209,600


**Task 3**
Convert the frequencies in task 2's cross-tab to percentages. Include the margins. (Hint: You can use apply)

In [31]:
df = pd.read_csv("bank-data.csv")

ct = pd.crosstab(df["save_act"], df["mortgage"], margins=True)

grand_total = ct.loc["All", "All"]
ct_percent = ct.apply(lambda col: (col / grand_total) * 100)

ct_percent


mortgage,NO,YES,All
save_act,,,
NO,20.166667,10.833333,31.0
YES,45.000000,24.000000,69.0
All,65.166667,34.833333,100.0
